<title> Building Calnet Example</title>
# How to build Calnet
written by Aric Sanders, 02/2016

A general description of how to build calnet. 

## 1. Make the skeleton 

   1. run "django-admin startproject MyProject" from a command prompt
   2. move into the directory containing manage.py 
        + i.e. run "cd MyProject"
   3. run "python manage.py startapp AppName" for each app
        + if you have multiple python installs you can replace python with the path to the bin or exe
   4. make some standard directories your favorite way
        + django name spaces its static directories as AppName/static/AppName/ the same with template directories
        
<em>Here we use a python script to create the directories we need</em>

In [1]:
import os
# First we define some functions to generate templated files

def write_project_urls(app_names):
    """Writes a basic urls.py for a django-project 
    given the app names"""
    urls_imports=['from django.conf.urls import include, url\n',
                 'from django.contrib import admin\n']
    
    url_pattern_lines=['urlpatterns= [\n',
                       "  url(r'^admin/', admin.site.urls),\n"
                      ]
    for app_name in app_names:
        url_pattern="  url(r'^%s/',include('%s.urls')),\n"%(app_name,app_name)
        url_pattern_lines.append(url_pattern)
    url_pattern_lines.append('  ]')

    out_file=open('urls.py','w')
    for line in urls_imports:
        out_file.write(line)
    for line in url_pattern_lines:
        out_file.write(line)
    out_file.close()
    
def write_app_urls(app_name):
    """Writes a basic urls.py for a django-app"""
    # Write the import string
    urls_imports=['from django.conf.urls import url\n',
                 'from . import views\n']
    # write the app_name = line, allows for namespacing the url template tags
    app_name_line="app_name= '%s'\n"%app_name
    # write the url patterns
    url_pattern_lines=['urlpatterns= [\n',
                       "  url(r'^$',views.index,name='index'),\n",
                      '  ]\n']
    # open the file and do the work
    out_file=open('urls.py','w')
    for line in urls_imports:
        out_file.write(line)
    out_file.write(app_name_line)
    for line in url_pattern_lines:
        out_file.write(line)
    out_file.close()
    

def write_admin_file(directory=None):
    """Writes an admin file in specified directory, defaults to the current directory"""
    if directory is not None:
        os.chdir(directory)
    admin_file_template=['# class functions from the standard library\n', 'import pyclbr\n', 'import re\n', 
                     'import os\n', 'from django.contrib import admin\n', '\n', 
                     'def register_all_models(module=None,path=None):\n', 
                     '    """ This function registers all modules in with the django admin. \n',
                     "    The module name should be a string, and defaults to 'models' and the path can be a string, list or tuple\n", '    If you include the admin.ModelAdmin in the models.py module with the same name + Admin\n', '    then it will register them too. Example if the model class is Pizza, then the admin model\n',
                     '    class would be PizzaAdmin """\n', '    if module is None:\n', "        module='models'\n", 
                     '    if path is None:\n', '        path=os.path.dirname(os.path.abspath(__file__))\n',
                     '        classes = pyclbr.readmodule(module,[path])\n', '    elif type(path) is str:\n', 
                     '        classes = pyclbr.readmodule(module,[path])\n', '    else:\n', 
                     '        classes = pyclbr.readmodule(module,path)\n', '    # first make a list of string only parents\n',
                     '    for model in classes:\n', '        if classes[model].super[0] in classes.values():\n', 
                     '            classes[model].super=classes[model].super[0].super\n', '\n', 
                     '    # make a list of admin classes\n', '    admin_classes=[]\n', '    for model in classes:\n', 
                     '        for superclass in classes[model].super:\n', '            try:\n', 
                     "                if re.search('admin.ModelAdmin',superclass):\n", 
                     '                    admin_classes.append(model)\n', 
                     '            except:pass\n', '    for model in classes:\n', 
                     '        # now the dirty part, check that the models are classes that inherit from models.Model\n',
                     '        # if this inhertance is not explicit in the class call it will not be registered\n',
                     '        for superclass in classes[model].super:\n', '            try:\n',
                     "                if re.search('models.Model',superclass):\n", 
                     '                    try:\n',
                     '                        # Check to see if the modelNameAdmin is in the list of admin classes\n',
                     "                        test_name=model+'Admin'\n", 
                     '                        if test_name in admin_classes:\n', 
                     "                            exec('from %s import %s,%s'%(module,model,test_name))\n", 
                     "                            exec('admin.site.register(%s,%s)'%(model,test_name))\n", 
                     '                        else:\n', 
                     '                        # this could be a from module import * above this loop\n',
                     "                            exec('from %s import %s'%(module,model))\n", 
                     "                            exec('admin.site.register(%s)'%model)\n", '                    except:raise\n', 
                     '            except:pass\n', 'register_all_models()']

    for line in admin_file_template:
        out_file.write(line)
    out_file.close()

In [3]:
try:
    # set the top most directory
    current_directory=os.getcwd() # Should change this if it needs to be
    # name the project
    project_name='Calnet'
    #name the apps, use a general naming scheme. Here I have used CamelCase and Nouns
    # Always include the home app, it will be where the base templates and views live
    app_names=['Home','Repository','ProjectTracker','CheckStandard',
               'Uncertainties','Measurement','Preferences','Help']
    # run a system command to create the project
    os.system('django-admin startproject %s'%project_name)
    # move into the directory with manage.py in it
    os.chdir(os.path.join(current_directory,'%s'%project_name))
    # keep the current directory
    current_directory=os.getcwd()
    manage_directory=os.getcwd() # we will want this directory for a lot of things
    # write a urls for the project
    os.chdir(os.path.join(manage_directory,'%s'%project_name))
    write_project_urls(app_names)
    os.chdir(manage_directory)
    # loop through the apps creating them 
    for app in app_names:
        os.system('python manage.py startapp %s'%app)   
    # Now write the urls.py for each app, and a general admin that imports all models from models into the admin
    for app in app_names:
        os.chdir(os.path.join(manage_directory,'%s'%app))
        write_app_urls(app)
        write_admin_file()
    # Once the apps are made, make static and 
    # template directories in each app for {{ static files }}
    # Note this could be combined with the 'startapp' loop
    for app in app_names:
        # static directory is redundant for makedirs, but retained for clarity
        # These directories are in the location django will look for them
        static_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app)
        template_directory=os.path.join(current_directory,'%s'%app,'templates','%s'%app)
        # This could be a list of exec commands
        img_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'img')
        js_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'js')
        css_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'css')
        # directories specific to my way of doing help or documentation
        jupyter_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'jupyter')
        html_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'html')
        cache=os.path.join(current_directory,'%s'%app,'static','%s'%app,'cache')
        application_data_directory=os.path.join(current_directory,'%s'%app,'static','%s'%app,'application_data')
        # help application
        if app in ['Help','help','doc','docs','documentation']:
            os.makedirs(static_directory)
            os.makedirs(template_directory)
            os.makedirs(img_directory)
            os.makedirs(js_directory)
            os.makedirs(css_directory)
            os.makedirs(cache)
            os.makedirs(jupyter_directory)
            os.makedirs(html_directory)
            
        elif app in ['CheckStandard']:
            os.makedirs(static_directory)
            os.makedirs(template_directory)
            os.makedirs(img_directory)
            os.makedirs(js_directory)
            os.makedirs(css_directory)
            os.makedirs(cache)
            os.makedirs(jupyter_directory)
            os.makedirs(html_directory)
            os.makedirs(application_data_directory)
        
            
        else:
            os.makedirs(static_directory)
            os.makedirs(template_directory)
            os.makedirs(img_directory)
            os.makedirs(js_directory)
            os.makedirs(css_directory)
            os.makedirs(cache)
except:raise

In [4]:
manage_directory

'C:\\Users\\arics\\Jupyter-Notebooks\\Calnet'

## 2. Define the models that you will need for each app
+ If you don't know yet don't worry
    1. Go to the App/models.py python file and add a model
        + Each model is a class that is derived from models.Model
        + Each model has Fields that are the columns
        + By Adding the \__str__ method you control how django displays the model
        + If you want to define a general "schema" or model form then use an inner class Meta, with the attribute abstract = True, I think these will be at pyMeasure.Code.DataHandlers.AbstractDjangoModels
        ```python
        class LogSchema(django.db.models.Model):
        """ A class schema that holds an index and a date """
        # This is an explicit replica of the auto generated field id
        index=IntegerField('Entry Number')
        # I use a date time field to time stamp entries
        # if you want to fix it to the point the entry is added then use ,auto_now_add=True
        date=DateTimeField('Date of entry')
        
        # The meta class is anything that is not a field. Setting proxy or abstract to True are special cases
        class Meta:
            abstract=True
        def __str__(self):
            """ A default setting override in new logs"""
            string_text="Entry %s at %s"%(self.index,str(self.date))
            return string_text
            ```
        + Now the actual table is the class that inherits from it
           ```python
           class UserLog(LogSchema):
           """ A Log that has a user and text field associated with it"""
           # I am not sure if this is supposed to be user or settings.AUTH_USER_MODEL
           user=models.ForeignKey(settings.AUTH_USER_MODEL,models.SET_NULL,blank=True,null=True,)
           # Now a text area or char field your choice, could be a summernote area also
           note=TextField('Note')
           def __str__(self):
               """ Display as a string, in this case Note number 1 left by user on date """
               # Todo replace with "".format()
               string_text="Note number %s left by %s %s on %s"%(str(self.index),User.first_name, User.last_name, str(self.date))
               return string_text
               ```
               

## 3. Register each model with App using the admin

1. Go to each Calnet/Help/admin.py and register all the models

```python
from django.contrib import admin

from .models import UserLog

admin.site.register(UserLog)
```
2. __UPDATED METHOD__ if you want to register all models, use the folowing code:

```python
# class functions from the standard library
import pyclbr #python class library, for determing classes inside a module
import re # python regular exprressions module
import os # python operating system module
from django.contrib import admin # django specific module for the admin site
def register_all_models(module=None,path=None):
    """ This function registers all modules in models with the django admin. 
    The module name should be a string, and defaults to 'models' and the path can be a string, list or tuple
    If you include the admin.ModelAdmin in the models.py module with the same name + Admin
    then it will register them too. Example if the model class is Pizza, then the admin model
    class would be PizzaAdmin. It can also deal with single class inheritance with in the models module. """
    
    # Assign the default module name if the value is None
    if module is None:
        module='models'
    # Assign the default path name if it is None
    if path is None:
        path=os.path.dirname(os.path.abspath(__file__)) #this is the robust way of getting the path of the module that is executing
        classes = pyclbr.readmodule(module,[path])  #note the pyclbr.readmodule requires a list or tuple as a path
    elif type(path) is str:
        classes = pyclbr.readmodule(module,[path])
    else:
        classes = pyclbr.readmodule(module,path)
    # first make a list of string only parents, this handles the casses in which pyclbr returns an object
    for model in classes:
        if classes[model].super[0] in classes.values():
            classes[model].super=classes[model].super[0].super

    # make a list of admin classes
    admin_classes=[]
    for model in classes:
        for superclass in classes[model].super:
            try:
                if re.search('admin.ModelAdmin',superclass):
                    admin_classes.append(model)
            except:pass
    for model in classes:
        # now the dirty part, check that the models are classes that inherit from models.Model
        # if this inhertance is not explicit in the class call in the module it will not be registered
        for superclass in classes[model].super:
            try:
                if re.search('models.Model',superclass):
                    try:
                        # Check to see if the modelNameAdmin is in the list of admin classes
                        test_name=model+'Admin'
                        if test_name in admin_classes:
                            exec('from %s import %s,%s'%(module,model,test_name))
                            exec('admin.site.register(%s,%s)'%(model,test_name))
                        else:
                        # this could be a from module import * above this loop
                            exec('from %s import %s'%(module,model))
                            exec('admin.site.register(%s)'%model)
                    except:raise
            except:pass
    # TODO: Check that models that inherit from another module work, check that 2 or more inheritace steps work
register_all_models() # Comment to run in notebook
```

## 4. Update the site/settings.py


1. Register the Apps, Open Calnet/Calnet/settings.py and change INSTALLED_APPS as follows:

    ```python
    INSTALLED_APPS = [
    'polls.apps.PollsConfig',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'Home',
    'Help',
    'Repository',
    'Preferences',
    'ProjectTracker',
    'CheckStandard',
    'Uncertainties',
    'Measurement'
    ]
    ```
2. Add MEDIA_ROOT and MEDIA_URL
    ```python 
     # At the very bottom of settings.py add these
    # MEDIA_ROOT is the physical storage location, must be different than STATIC_ROOT
    MEDIA_ROOT ='/Path_to_Data_Drive'
    # MEDIA_URL is the URL that maps to MEDIA_ROOT, note you must  add 'django.template.context_processors.media' in the 'context_processors' option of TEMPLATES to use {{ MEDIA_URL }}.
    MEDIA_URL ='/media/'
      ```
  
3. Configure TEMPLATES

    ```python
    TEMPLATES = [
                    {
                        'BACKEND': 'django.template.backends.django.DjangoTemplates',
                        'DIRS': [],
                        'APP_DIRS': True,
                        'OPTIONS': {
                            'context_processors': [
                                'django.template.context_processors.debug',
                                'django.template.context_processors.request',
                                'django.contrib.auth.context_processors.auth',
                                'django.contrib.messages.context_processors.messages',
                                'django.template.context_processors.media',
                            ],
                        },
                    },
                ]
    ```
4. Configure the static files settings

    ```python
        # Make sure the static files are configured correctly
        # STATIC_URL is how the site references the static direcories
        STATIC_URL = '/static/'
        # when depolyed the static files are served by Apache so this is very important
        STATIC_ROOT = 'PATH_TO_COLLECTION_POINT_FOR_ALL_STATIC_FILES'
        # This should ultimately point to any place outside of the Project and App directories that has static files
        STATICFILES_DIRS = [
        "/Path to common static files/",
           ] # TODO make this point to pyMeasure.Code.FrontEnds.?.FRONTENDS_DIRECTORY/js etc... and static root
       ```
5. Set the site id

    ```python 
    # This allows other apps and sites to talk with each other
    SITE_ID='Calnet'
    ```
    
           


    

## 5. Make Migrations

1. In order to update the models in the database and with the admin site you have to run makemigrations and migrate.

In [ ]:
# make sure you are in the manage_directory
#import os if a different session from the skeleton generation
os.chdir(manage_directory)
os.system('python manage.py makemigrations')
os.system('python manage.py migrate')

## 6. Create a superuser account

1. In order to create a super user account use the shell to run python manage.py createsuperuser
    + Note if you have some dead references such as importing a view that is not defined yet then this will not work, I think this should be ran after the skeleton and make sure the skeleton is compatible..
```python
# This should work, not sure if the shell will stop
os.system('python manage.py createsuperuser')
```


## 7. Make urls.py in the site directory point to the App urls and admin

1. Go to the MySite/urls.py and add each App
    + __I actually took care of this in the skeleton generation__

```python
from django.conf.urls import include, url
from django.contrib import admin

urlpatterns = [
    url(r'^App/', include('App.urls')),
    url(r'^admin/', admin.site.urls),
]
```

## 8. Make the App urls point to views

1. Open each App/urls.py and put a pattern that points to a view
    + __I made a default view, index, for every app in the skeleton generation__

## 9. Define the views in App/view.py

1. open each Calnet/App/views.py and add a function that returns a view, use templates.
    + I need to add the view index to each APP in the skeleton generation step

## 10. Define the templates in App/templates/App
    1. There should be an index.html template for the home app at site/home/templates/home/index.html
    2. Each app template should extend this one i.e {% extends 'index.html' %}

## 11. Add all the needed javascript, images and css to App/static/App/

1. Does css and js need to be repeated? I think the collect static fixes this for the site